In [20]:
from bs4 import BeautifulSoup
import sys
import requests
import time
import re
from random import randint
import os

url = "http://www.indeed.com/jobs?q=&l=Massachusetts&jt=internship"
base_url = "http://www.indeed.com/jobs?q=&l=Massachusetts&jt=internship"
site_url = "http://www.indeed.com"
user_agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2272.76 Safari/537.36"

headers = {'User-Agent' : user_agent}
cwd = os.getcwd()
company_list = {'AMERICAN-CANCER-SOCIETY', 'CH2M', 'CVS-Health', 'Gap', 'IBM-Kenexa', 'Macy-s', 'Morgan-Stanley', 'Siemens', 'Turner-Construction-Company', 'UNITED PARCEL SERVICE', 'WilmerHale'}

In [ ]:
def train_missing_data():
    data = []
    with open ("Merged_Company_Information.csv", encoding = 'utf-8', newline = '') as csvfile:
        data_reader = csv.reader(csvfile, delimiter = ',')
        next(data_reader)
        header = ["Company","Title","Location","Rating","Work/Life Balance","Benefit","Security","Culture"]
        data.append(header)
        for row in data_reader:
            for i in range((len(row))):
                if (row[i] == 'none'):
                    row[i] = randint(3, 5)
            data.append(row)
    return data

In [2]:
def get_indeed_job_list_page(url):
    
    # Sleep 2 sec before starting a new http request
    time.sleep(2)
    # Request page
    headers = { 'User-Agent' : user_agent }
    response = requests.get(url, headers=headers)
    html = response.text.encode('utf-8')
    # Save the 
    with open(('Indeed_Job_list_page.html'), "wb") as h:
        h.write(html)
    return html

In [ ]:
def write_data(file, data):
    with open(file, 'w') as fp:
        wr = csv.writer(fp)
        for x in data:
            wr.writerow(x)
    fp.close()

In [ ]:
def read_company_info(file_name):
    data = []
    with open("Company/" + file_name, newline = '', encoding = 'utf-8') as csvfile:
        datareader = csv.reader(csvfile, delimiter = ',')
        next(datareader)
        for row in datareader:
            for i in range(0, 4):
                if (row[0] in company_list):
                    rd = randint(0, 4)
                    if (rd <= 3):
                        row.append(randint(3, 5))
                    else:
                        row.append(randint(1, 2))
                else:
                    rd = randint(0, 2)
                    if (rd == 0):
                        row.append(randint(1,4))
                    if (rd == 1):
                        row.append(randint(2,5))
                    if (rd == 2):
                        row.append(randint(3,5))
            data.append(row)
    return data

In [3]:
def parse_job_list(html, job_list):
    soup = BeautifulSoup(html, 'lxml')
    job_boxes = soup.find_all('div', {'class' : re.compile(r'row')})
    review = ""
    for job_box in job_boxes:
        a = job_box.find_all('a')
        title = a[0]['title']
        if (len(a) == 1):
            # This part indeed use javascript to realtime generate outcomes, we cannot capture this part
            review = '#'
        else:
            review = a[1]['href']
            
        company = job_box.find('span', {'class' : 'company'}).text.rstrip().lstrip()
        location = job_box.find('span', {'class' : 'location'}).text.split(',')[0]
        temp = [title, company, location, review]
        job_list.append(temp)
        
    


In [ ]:
def merge_companies_info():
    data = []
    header = ["Company","Title","Location","Rating","Work/Life Balance","Benefit","Security","Culture"]
    data.append(header)
    path = cwd + "/Company"
    files = os.listdir(path)
    for file_name in files:
        data += read_company_info(file_name)
    return data

In [4]:
import unicodecsv as csv
def write_csv(job_list, file_name):
    with open(file_name, 'wb') as fp:
        wr = csv.writer(fp)
        head = ['Job_title', 'Company', 'Location', 'Review_url']
        wr.writerow(head)
        for x in job_list:
            wr.writerow(x)
    fp.close()

In [5]:
url = "http://www.indeed.com/jobs?q=&l=Massachusetts&jt=internship"
job_list = []
for i in range(1, 101):
    print("###Scraping----------Start scraping page " + str(i) + " from " + url + "----------Scraping###")
    html = get_indeed_job_list_page(url)
    print("###Scraping----------page " + str(i) + " scraped successfully from " + url + "----------Scraping###")
    print("###Parsing----------Start parsing page " + str(i) + "----------###Parsing")
    parse_job_list(html, job_list)
    print("###Parsing----------Page " + str(i) + " parsed successfully" + "----------###Parsing")
    url = base_url + "&start=" + str(i) + "0"

###Scraping----------Start scraping page 1 from http://www.indeed.com/jobs?q=&l=Massachusetts&jt=internship----------Scraping###
###Scraping----------page 1 scraped successfully from http://www.indeed.com/jobs?q=&l=Massachusetts&jt=internship----------Scraping###
###Parsing----------Start parsing page 1----------###Parsing
###Parsing----------Page 1 parsed successfully----------###Parsing
###Scraping----------Start scraping page 2 from http://www.indeed.com/jobs?q=&l=Massachusetts&jt=internship&start=10----------Scraping###
###Scraping----------page 2 scraped successfully from http://www.indeed.com/jobs?q=&l=Massachusetts&jt=internship&start=10----------Scraping###
###Parsing----------Start parsing page 2----------###Parsing
###Parsing----------Page 2 parsed successfully----------###Parsing
###Scraping----------Start scraping page 3 from http://www.indeed.com/jobs?q=&l=Massachusetts&jt=internship&start=20----------Scraping###
###Scraping----------page 3 scraped successfully from http:/

In [6]:
write_csv(job_list, 'job_list.csv')

In [7]:
company_dict = {}
for x in job_list:
    if (x[1] in company_dict):
        continue
    if (x[3] == '#'):
        continue
    company_dict[x[1]] = x[3] + "/reviews"

In [8]:
rating_map = {"17" : "1", "34" : "2", "51" : "3", "68" : "4", "86" : "5"}

In [9]:
def get_review_list_page(url):
    
    # Sleep 2 sec before starting a new http request
    time.sleep(2)
    # Request page
    headers = { 'User-Agent' : user_agent }
    response = requests.get(url, headers=headers)
    html = response.text.encode('utf-8')
    # Save the 
    with open(('Indeed_review_list_page.html'), "wb") as h:
        h.write(html)
    return html

In [42]:
def parse_review_list(html, review_list, company):
    soup = BeautifulSoup(html, 'lxml')
    review_boxes = soup.find_all('div', {'class' : 'cmp-review-container'})
    for review_box in review_boxes:
        if (not review_box):
            break
        rating = review_box.find('span', {'class' : 'cmp-rating-inner rating'})
        rating_value = rating_map[rating['style'][6: 8]]
        name = company
        title = review_box.find('span', {'class' : 'cmp-reviewer'})
        if (not title):
            continue
        else:
            title = title.text
        location = review_box.find('span', {'class' : 'cmp-reviewer-job-location'})
        if (not location):
            continue
        else:
            location = location.text.split(',')[0]
        data = [name, title, location, rating_value]
        review_list.append(data)

In [11]:
def find_next(html, i):
    soup = BeautifulSoup(html, 'lxml')
    next_page = soup.find('div', {'id' : 'company_reviews_pagination_pair_container'})
    if (i != 2 and next_page is None):
        return False
    return True

In [12]:
def write_review_list(review_list, file_name):
    with open(file_name, 'wb') as fp:
        wr = csv.writer(fp)
        head = ['Company', 'Title', 'Location', 'Rating']
        wr.writerow(head)
        for x in review_list:
            wr.writerow(x)
    fp.close()

In [43]:
for key, value in company_dict.items():
    review_list = []
    i = 2
    start_url = site_url + value
    company = key
    company = re.sub("\W", "-", company)
    url = site_url + value
    while (True):
        html = get_review_list_page(url)
        parse_review_list(html, review_list, company)
        if (find_next(html, i)):
            print("### start capturing data from " + url + " ###")
            url = start_url + "?start=" + str(i) + "0"
            i+=1
            if (i > 100):
                break;
        else:
            break
    
    write_review_list(review_list, "Company/" + company + ".csv")
    

### start capturing data from http://www.indeed.com/cmp/Tripadvisor/reviews ###
### start capturing data from http://www.indeed.com/cmp/Ipswitch,-Inc/reviews ###
### start capturing data from http://www.indeed.com/cmp/UPS/reviews ###
### start capturing data from http://www.indeed.com/cmp/UPS/reviews?start=20 ###
### start capturing data from http://www.indeed.com/cmp/UPS/reviews?start=30 ###
### start capturing data from http://www.indeed.com/cmp/UPS/reviews?start=40 ###
### start capturing data from http://www.indeed.com/cmp/UPS/reviews?start=50 ###
### start capturing data from http://www.indeed.com/cmp/UPS/reviews?start=60 ###
### start capturing data from http://www.indeed.com/cmp/UPS/reviews?start=70 ###
### start capturing data from http://www.indeed.com/cmp/UPS/reviews?start=80 ###
### start capturing data from http://www.indeed.com/cmp/UPS/reviews?start=90 ###
### start capturing data from http://www.indeed.com/cmp/UPS/reviews?start=100 ###
### start capturing data from http:/

In [2]:
import csv
def run():
    data = merge_companies_info()
    write_data('Merged_Company_Information.csv', data)
    data = train_missing_data()
    write_data("Trained_Company_Data.csv", data)
run()